<a href="https://colab.research.google.com/github/bnarath/TF_Developer/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we are going to cover some of the most fundamental concepts of tensors using TF 2.0

Topics being covered
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Unsing @tf.function(a way to speed up regular python functions)
* Using GPUs/TPUs with tensorflow
* Exercises


Methods
- tf.constant(tensor, shape, dtype) - It is an EagerTensor
- tf.Variable(tensor-initial, shape, dtype)
  - Assign values to variables by .assign()
- tf.random.Generator.from_seed(100)
  - tf.random(shape, minval, maxval, dtype)
  - tf.normal(shape, dtype)

- tf.random.set_seed(10)
  - tf.random.shuffle(not_shuffled)


Information
- tensor.ndim (len(shape))
- tensor.shape
- tf.size(tensor) - # of all elements
- Indexing (same as numpy indexing)
- tensor.shape[-1]
- tensor.numpy()


Add extra dimension
- tensor[ :, :, tf.newaxis] (tf.newaxis is same as None in numpy)
- tensor[ ..., tf.newaxis]
- tf.reshape(tensor, (tensor.shape[0], tensor.shape[1], 1))
- tf.expand_dims(tensor, -1)


Manipulating Tensors (Tensor Operations)

- Broadcasting Rules
  - Lower dimensional tensor extends to get dimension same as the higher
  - In each dimension, lengths should either match between 2 tensors or extendible
 
 - Multiply
    - tf.multiply (element wise) (alias of tf.math.multiply)
    - A*b (element wise)

- Add
  - tf.add (alias of tf.math.add)

- Matrix multiplication
  - AxB[i, j] is the dot product of ith row of A and jth column of B
  - tf.matmul
  - A @ B
  - tf.tensordot(A, B, axes=[[1], [0]]) - Axes defines vectors considered in dot product

- Transpose
  - tf.transpose

Changing datatype of tensors

- Default Precision
  - float32, int32

- Mixed Precision
  - lower bit float16/bfloat16 are better with performance, gpus and tpus are better with these
  - higher bit float32 is good for numeric stability
  - mixed precisions can be used in training 

- Change dtype
  - tf.cast(A, dtype=tf.float16)


- Aggregation
  - tf.reduce_min(tensor, axis=1)
  - tf.reduce_max
  - tf.reduce_mean
  - tf.reduce_sum
  - tf.abs(A) : absolute value of each element
  - tfp.stats.variance(A,sample_axis=None) : From tensorflow-probability
  - tf.math.reduce_std(A)
  - tf.math.argmax(A, axis=0)
  - tf.math.argmin(A, axis=0)


- Squeeze
  - tf.squeeze (removes dimensions of shape 1)


- One hot encoding
  ```items = ['red', 'blue', 'green']
      indices = range(len(items))
      tf.one_hot(indices, depth=len(indices))```

- A few math operations
  - tf.range(1, 10, dtype=tf.float32)
  - tf.square(a)
  - tf.math.sqrt(a)
  - tf.math.log(a) (natural logarithm : https://stackoverflow.com/questions/3719631/log-to-the-base-2-in-python)


- Compatibility with numpy array
  - tensors can be created from numpy arrays and vice-versa. Default data types might not match between both and depends on versions

  - **One of the main difference - tensors are faster in gpu and tpu**


- **GPU availability**
  - check physicl devices : 
    - tf.config.list_physical_devices()
    - tf.config.list_physical_devices("TPU")
    - tf.config.list_physical_devices("GPU")
    - tf.config.list_physical_devices("CPU")

  - Nvidia system management utility command to check GPU
    - !nvidia-smi
      - Gives smi version
      - GPU driver version
      - CUDA version (interface between tf code and gpu)
      - Type of GPU (Eg: Tesla T4)
      - Memory usage (out of 15 GB memory)
      - Cannot choose the GPU type
      - **If we have access to CUDA enabled GPU, TF automatically uses it**

- Assertion
  - tf.Assert: 
    - argumnents :
      -  condition (1dim tensor)
      - data: tensors part of condition
      - summarize: the depth in terms of #elements to be included in error code. 
    - Returns: If assertion is working, none output
    ```
    a = tf.constant([1.,2.,3.])
    b = tf.constant([1.,2.,4.])
    tf.Assert(tf.greater_equal(tf.math.count_nonzero(a==b, dtype=tf.int32), tf.size(a)), [a, b], summarize=tf.size(a))
    ```


## Introduction to Tensors

In [ ]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.9.2'

## Create tensors using tf.Constant

In [ ]:
scalar = tf.constant(9, dtype='int16')
scalar.ndim

0

In [ ]:
vector = tf.constant([1,2,3])
vector, vector.ndim

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>, 1)

In [ ]:
matrix = tf.constant([[1.,2], [3,4]])
matrix, matrix.ndim

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>, 2)

In [ ]:
nd_matrix=tf.constant(np.arange(1, 243, 3), shape=(3,3,3,3))
nd_matrix.ndim

4

# What is a Tensor

n domensional array of numbers 
- n=0, scalar
- n=1, vector
- n=2, metrix


## Create tensors using tf.Variable

The Variable() constructor requires an initial value for the variable, which can be a Tensor of any type and shape. This initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods.

In [ ]:
x = tf.Variable([1, 2])
x

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>

In [ ]:
x.assign([2,3])
x[1].assign(10)
x

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([ 2, 10], dtype=int32)>

# Create random tensors

In [ ]:
random1=tf.random.Generator.from_seed(100)
random1.uniform(
    (10,),
    minval=0,
    maxval=10,
    dtype=tf.int32
)
random1.normal(
    (10,),
    dtype=tf.float16
)

<tf.Tensor: shape=(10,), dtype=float16, numpy=
array([-0.785 ,  0.0869, -0.3286,  0.4028, -0.9795, -0.2349, -0.462 ,
        2.408 , -0.6445,  0.6567], dtype=float16)>

In [ ]:
random2=tf.random.Generator.from_seed(100)
random2.uniform(
    (10,),
    minval=0,
    maxval=10,
    dtype=tf.int32
)
random2.normal(
    (10,),
    dtype=tf.float16
)

<tf.Tensor: shape=(10,), dtype=float16, numpy=
array([-0.785 ,  0.0869, -0.3286,  0.4028, -0.9795, -0.2349, -0.462 ,
        2.408 , -0.6445,  0.6567], dtype=float16)>

# Shuffle the order of elements in a tensor

In [ ]:
not_shuffled = tf.constant([[10,7],
                            [1,2],
                            [2,4]])

In [ ]:
tf.random.set_seed(10)
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  4],
       [ 1,  2]], dtype=int32)>

In [ ]:
tf.random.set_seed(10)
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  4],
       [ 1,  2]], dtype=int32)>

## ⚓ Exercise: Produce 5 random tensors and shuffle them

https://www.tensorflow.org/api_docs/python/tf/random/set_seed

In [ ]:
tf.random.set_seed(100)
random_gen = tf.random.Generator.from_seed(10)
a = random_gen.normal(shape=(1,10), mean=0, stddev=1)
b = random_gen.uniform(shape=(1,10), minval=0, maxval=10)
c = tf.reshape(
    tf.random.stateless_binomial(
    shape=[10], seed=[1, 1], counts=[10]*10, probs=np.arange(0.1, 1.1, 0.1), output_dtype=tf.float32), (1,10))
d = random_gen.normal(shape=(1,10), mean=10, stddev=1)
e = random_gen.normal(shape=(1,10), mean=0, stddev=2)
unshuffled = tf.concat([a,b,c,d,e], 0)
unshuffled, tf.random.shuffle(unshuffled, seed=10)

(<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
 array([[-2.9604465e-01, -2.1134207e-01,  1.0630016e-02,  1.5165398e+00,
          2.7305740e-01, -2.9925638e-01, -3.6523250e-01,  6.1883307e-01,
         -1.0130816e+00,  2.8291711e-01],
        [ 4.0812969e-01,  1.1846912e+00,  8.4382048e+00,  7.6974926e+00,
          6.6823254e+00,  3.8150179e+00,  3.5973561e+00,  3.4344697e+00,
          8.9352741e+00,  5.9334860e+00],
        [ 3.0000000e+00,  2.0000000e+00,  1.0000000e+00,  6.0000000e+00,
          2.0000000e+00,  5.0000000e+00,  6.0000000e+00,  8.0000000e+00,
          8.0000000e+00,  1.0000000e+01],
        [ 8.8429270e+00,  1.0771253e+01,  1.0319363e+01,  1.0404154e+01,
          1.0327567e+01,  1.1497083e+01,  9.8136797e+00,  1.2315570e+01,
          1.0519485e+01,  9.1210108e+00],
        [-1.6040002e+00, -4.1129370e+00,  1.0339310e+00,  2.2310841e+00,
          2.2762547e+00,  4.2137284e+00,  1.8501984e+00, -4.0433061e-01,
          2.8731544e+00,  7.6397896e-02]], dtype=flo

# Other ways to create tensors

In [ ]:
print(tf.ones((1,10)))
tf.zeros((1,10))

tf.Tensor([[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(1, 10), dtype=float32)


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

# Turning numpy arrays into tensors

In [ ]:
np_A = np.arange(1, 25, dtype=np.int32)
A = tf.constant(np_A, shape=(2,3,4))
B = tf.constant(np_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

# Getting information from tensors

In [ ]:
A = tf.zeros(shape=(2, 3, 4, 5))
A.ndim, A.shape, tf.size(A), A[-1, -1, -1], A.dtype

(4,
 TensorShape([2, 3, 4, 5]),
 <tf.Tensor: shape=(), dtype=int32, numpy=120>,
 <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 tf.float32)

# Indexing Tensors

- like lists or numpy

In [ ]:
random_gen = tf.random.Generator.from_seed(10)
A = random_gen.uniform((2,3,4,5), minval=0, maxval=10, dtype=tf.int32)

In [ ]:
#Get first 2 elements of eatch dimension
print(A, A[:2, :2, :2, :2])

#Get first element in each dimension except the final one
print(A[0, 0, 0, :])

#Get first element in each dimension except the second last one
print(A[:1, :1, :, :1])

tf.Tensor(
[[[[6 8 2 4 4]
   [4 1 7 0 9]
   [2 9 3 9 6]
   [6 4 2 5 7]]

  [[9 6 9 1 3]
   [1 0 4 7 2]
   [5 3 0 3 6]
   [4 7 1 3 6]]

  [[1 7 4 9 0]
   [5 1 8 6 9]
   [8 1 8 2 9]
   [9 6 6 9 6]]]


 [[[3 0 3 2 5]
   [9 2 3 6 5]
   [7 0 2 6 2]
   [5 6 1 2 3]]

  [[8 0 4 9 7]
   [0 2 2 3 4]
   [8 3 7 9 3]
   [7 1 5 7 3]]

  [[1 7 5 0 7]
   [2 0 1 1 8]
   [2 3 8 9 6]
   [3 9 9 4 9]]]], shape=(2, 3, 4, 5), dtype=int32) tf.Tensor(
[[[[6 8]
   [4 1]]

  [[9 6]
   [1 0]]]


 [[[3 0]
   [9 2]]

  [[8 0]
   [0 2]]]], shape=(2, 2, 2, 2), dtype=int32)
tf.Tensor([6 8 2 4 4], shape=(5,), dtype=int32)
tf.Tensor(
[[[[6]
   [4]
   [2]
   [6]]]], shape=(1, 1, 4, 1), dtype=int32)


# Add extra dimension to tensor

In [ ]:
rank_2_tensor = random_gen.uniform((2,2), minval=0, maxval=10, dtype=tf.int32)

In [ ]:
rank_2_tensor.shape

TensorShape([2, 2])

In [ ]:
rank_2_tensor[ :, :, tf.newaxis]
rank_2_tensor[ ..., tf.newaxis]
tf.reshape(rank_2_tensor, (rank_2_tensor.shape[0], rank_2_tensor.shape[1], 1))
tf.expand_dims(rank_2_tensor, -1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[2],
        [8]],

       [[3],
        [1]]], dtype=int32)>

# Tensor Manipulations (Tensor operations)

In [ ]:
A = random_gen.uniform((2,2), minval=0, maxval=10, dtype=tf.int32)
b = tf.ones((2, 1), dtype=tf.int32)
A, b

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[3, 0],
        [7, 7]], dtype=int32)>, <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[1],
        [1]], dtype=int32)>)

In [ ]:
A+10, A+b, A*b

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[13, 10],
        [17, 17]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[4, 1],
        [8, 8]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[3, 0],
        [7, 7]], dtype=int32)>)

In [ ]:
tf.multiply(A, b)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 0],
       [7, 7]], dtype=int32)>

In [ ]:
A * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[30,  0],
       [70, 70]], dtype=int32)>

# Matrix multiplication

AxB[i, j] is the dot product of ith row of A and jth column of B

In [ ]:
A = tf.Variable([[1, 2, 5], [7, 2, 1], [3, 3, 3]], dtype=tf.int32)
B = tf.constant([[3, 5], [6, 7], [1, 8]], dtype=tf.int32)

In [ ]:
tf.math.equal(tf.matmul(A, B), A @ B)
tf.equal(A@B, tf.tensordot(A, B, [[1], [0]]))

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

# Changing datatype of tensor

In [ ]:
tf.__version__, tf.constant([1., 2.]), tf.constant([1, 2])

('2.9.2',
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>)

In [ ]:
a = tf.constant([1., 2.])
a, a.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>,
 tf.float32)

In [ ]:
tf.cast(a, dtype=tf.float16), tf.cast(a, dtype=tf.bfloat16)

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1., 2.], dtype=float16)>,
 <tf.Tensor: shape=(2,), dtype=bfloat16, numpy=array([1, 2], dtype=bfloat16)>)

In [ ]:
b = tf.constant([1, 2])
tf.cast(b, dtype=tf.float32), tf.cast(a, dtype=tf.float32)

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>)

# Aggregating tensors

In [ ]:
random_gen = tf.random.Generator.from_seed(100)
A = random_gen.uniform(shape=(4,4), minval=-10, maxval=10, dtype=tf.int32)

In [ ]:
A, tf.abs(A), tf.reduce_min(A), tf.reduce_max(A, axis=0), tf.reduce_mean(A), tf.reduce_sum(A, axis=1)

(<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
 array([[ 0,  2,  1, -2],
        [-7, -8,  7, -4],
        [ 4,  7,  2,  9],
        [ 4,  8,  4, -3]], dtype=int32)>,
 <tf.Tensor: shape=(4, 4), dtype=int32, numpy=
 array([[0, 2, 1, 2],
        [7, 8, 7, 4],
        [4, 7, 2, 9],
        [4, 8, 4, 3]], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=-8>,
 <tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 8, 7, 9], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(4,), dtype=int32, numpy=array([  1, -12,  22,  13], dtype=int32)>)

In [ ]:
(A  - tf.reduce_mean(A)) * (A  - tf.reduce_mean(A))

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 1,  1,  0,  9],
       [64, 81, 36, 25],
       [ 9, 36,  1, 64],
       [ 9, 49,  9, 16]], dtype=int32)>

In [ ]:
A = tf.cast(A, dtype=tf.float32)
var = tf.reduce_mean(tf.square((A  - tf.reduce_mean(A))))
std = tf.sqrt(var)
var, std

(<tf.Tensor: shape=(), dtype=float32, numpy=25.375>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.03736>)

In [ ]:
import tensorflow_probability as tfp
tfp.stats.variance(A,sample_axis=None), tf.math.reduce_std(A)

(<tf.Tensor: shape=(), dtype=float32, numpy=25.375>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.03736>)

In [ ]:
A

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 0.,  2.,  1., -2.],
       [-7., -8.,  7., -4.],
       [ 4.,  7.,  2.,  9.],
       [ 4.,  8.,  4., -3.]], dtype=float32)>

In [ ]:
tf.math.argmax(A, axis=0), tf.math.argmin(A, axis=0)

(<tf.Tensor: shape=(4,), dtype=int64, numpy=array([2, 3, 1, 2])>,
 <tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 1, 0, 1])>)

In [ ]:
A[tf.math.argmax(A, axis=0)[1], 1]

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

# Squeezing a Tensor

In [ ]:
random_gen = tf.random.Generator.from_seed(100)
A = random_gen.uniform(shape=(1,2,1,3), minval=0, maxval=10)

In [ ]:
tf.squeeze(A)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[6.661966 , 7.469058 , 8.769325 ],
       [8.477416 , 2.2709787, 4.9256873]], dtype=float32)>

# One hot encoding 

In [ ]:
items = ['red', 'blue', 'green']
indices = range(len(items))
tf.one_hot(indices, depth=len(indices))

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

# Some math operations

In [ ]:
a = tf.range(1, 10, dtype=tf.float32)
a, tf.square(a), tf.math.sqrt(a), tf.math.log(a)

(<tf.Tensor: shape=(9,), dtype=float32, numpy=array([1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=array([ 1.,  4.,  9., 16., 25., 36., 49., 64., 81.], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
        2.6457512, 2.828427 , 3.       ], dtype=float32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
        1.9459102, 2.0794415, 2.1972246], dtype=float32)>)

# Compatibility with numpy array

In [ ]:
a=[1.,2.,3.]
a_np = tf.constant(a)
n_tensor1 = tf.constant(a)
n_tensor2 = tf.Variable(a)
a, a_np, n_tensor1, n_tensor2

([1.0, 2.0, 3.0],
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>)

# Access to gpu

- If we have access to CUDA enabled GPU, TF automatically uses it

In [ ]:
tf.config.list_physical_devices("TPU"), tf.config.list_physical_devices("GPU"), tf.config.list_physical_devices("CPU")

([],
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')],
 [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')])

In [ ]:
!nvidia-smi

Mon Jan 23 01:28:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W /  70W |    314MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 🛠 00. TensorFlow Fundamentals Exercises

1. Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().
1. Find the shape, rank and size of the tensors you created in 1.
1. Create two tensors containing random values between 0 and 1 with shape [5, 300].
1. Multiply the two tensors you created in 3 using matrix multiplication.
1. Multiply the two tensors you created in 3 using dot product.
1. Create a tensor with random values between 0 and 1 with shape [224, 224, 3].
1. Find the min and max values of the tensor you created in 6 along the first axis.
1. Created a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3].
1. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum value.
1. One-hot encode the tensor you created in 9.

In [ ]:
#1
scalar = tf.constant(10)
vector = tf.constant([1,2,3])
matrix = tf.constant(np.arange(1, 10), shape=(3,3))
scalar, vector, matrix

(<tf.Tensor: shape=(), dtype=int32, numpy=10>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,
 <tf.Tensor: shape=(3, 3), dtype=int64, numpy=
 array([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])>)

In [ ]:
#2
scalar.shape, vector.ndim, tf.size(matrix)

(TensorShape([]), 1, <tf.Tensor: shape=(), dtype=int32, numpy=9>)

In [ ]:
#3
random_gen = tf.random.Generator.from_seed(100)
A=random_gen.uniform((5, 300), minval=0, maxval=1)
B=random_gen.uniform((5, 300), minval=0, maxval=1)
A, B

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.6661966 , 0.7469058 , 0.8769325 , ..., 0.41536427, 0.4722309 ,
         0.2039113 ],
        [0.00572646, 0.5180874 , 0.18395662, ..., 0.02066433, 0.15314782,
         0.979172  ],
        [0.8016473 , 0.4288255 , 0.6153476 , ..., 0.5761353 , 0.247504  ,
         0.15598488],
        [0.88497293, 0.95242417, 0.7611464 , ..., 0.04600775, 0.37063193,
         0.40530336],
        [0.34529388, 0.2566265 , 0.6553159 , ..., 0.73266625, 0.3230319 ,
         0.3916204 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.6423917 , 0.66857696, 0.03149796, ..., 0.33779883, 0.40777087,
         0.9821553 ],
        [0.9981669 , 0.03780842, 0.23435211, ..., 0.43565345, 0.38620996,
         0.82172513],
        [0.884037  , 0.66895485, 0.7227589 , ..., 0.5935743 , 0.24899626,
         0.14523578],
        [0.63517976, 0.9365325 , 0.03962076, ..., 0.99744785, 0.95818496,
         0.4663638 ],
        [0.71846

In [ ]:
#4
A@tf.transpose(B)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[72.15505 , 72.07376 , 72.49989 , 74.66458 , 77.83046 ],
       [74.090775, 70.17688 , 74.07868 , 73.94577 , 73.172844],
       [72.373245, 71.25055 , 75.43869 , 74.33907 , 75.48618 ],
       [75.03104 , 74.18742 , 73.292366, 76.8454  , 76.95524 ],
       [78.20044 , 73.97373 , 76.448654, 75.0536  , 76.4449  ]],
      dtype=float32)>

In [ ]:
#5
tf.tensordot(A, B, axes=[[1], [1]])

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[72.15505 , 72.07376 , 72.49989 , 74.66458 , 77.83046 ],
       [74.090775, 70.17688 , 74.07868 , 73.94577 , 73.172844],
       [72.373245, 71.25055 , 75.43869 , 74.33907 , 75.48618 ],
       [75.03104 , 74.18742 , 73.292366, 76.8454  , 76.95524 ],
       [78.20044 , 73.97373 , 76.448654, 75.0536  , 76.4449  ]],
      dtype=float32)>

In [ ]:
#6
C = random_gen.uniform( [224, 224, 3], 0, 1)
C

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[8.03045273e-01, 3.63185406e-01, 2.61659741e-01],
        [9.61764336e-01, 3.41106653e-02, 2.91288137e-01],
        [5.79285622e-02, 9.88239527e-01, 4.18271661e-01],
        ...,
        [7.22702384e-01, 7.58950710e-01, 7.81779766e-01],
        [5.30807018e-01, 4.72644925e-01, 5.75111151e-01],
        [6.51104093e-01, 9.93840337e-01, 9.54247355e-01]],

       [[7.53824234e-01, 5.26019335e-01, 8.05311084e-01],
        [2.38019824e-01, 7.43054152e-02, 5.27126789e-02],
        [4.37600732e-01, 9.93556261e-01, 7.43962526e-02],
        ...,
        [8.36582422e-01, 6.32904410e-01, 8.68649483e-01],
        [4.53148961e-01, 3.72526050e-01, 9.66543317e-01],
        [5.62302470e-01, 9.94206667e-01, 3.13256979e-01]],

       [[8.55053663e-02, 1.19435072e-01, 3.08908820e-01],
        [8.54358673e-02, 2.17479348e-01, 7.90752172e-01],
        [8.72100592e-02, 1.73055887e-01, 9.01940107e-01],
        ...,
        [9.30227637e-01, 8.36251

In [ ]:
#7
tf.reduce_min(C, axis=0), tf.reduce_max(C, axis=0)

(<tf.Tensor: shape=(224, 3), dtype=float32, numpy=
 array([[8.03577900e-03, 5.93090057e-03, 7.93075562e-03],
        [2.69861221e-02, 1.75893307e-03, 3.88789177e-03],
        [5.76138496e-04, 9.68933105e-04, 5.70654869e-04],
        [1.65390968e-03, 2.35438347e-03, 3.73923779e-03],
        [2.61938572e-03, 5.02514839e-03, 3.65948677e-03],
        [6.19268417e-03, 9.94920731e-04, 5.87201118e-03],
        [3.24356556e-03, 1.03056431e-03, 4.06718254e-03],
        [7.02857971e-04, 2.75611877e-03, 1.04262829e-02],
        [3.85224819e-03, 2.21724510e-02, 1.42960548e-02],
        [1.05142593e-04, 3.34179401e-03, 2.85255909e-03],
        [1.53480768e-02, 3.85880470e-04, 2.16376781e-03],
        [1.79803371e-03, 1.54364109e-03, 5.40244579e-03],
        [1.00946426e-03, 3.19933891e-03, 1.89685822e-03],
        [1.31753683e-02, 7.67946243e-04, 9.46521759e-05],
        [1.03199482e-03, 7.21931458e-03, 9.94741917e-03],
        [1.34682655e-03, 4.68790531e-03, 2.56133080e-03],
        [2.52187252e-

In [ ]:
#8
D = random_gen.uniform( [1, 224, 224, 3], 0, 1)
tf.squeeze(D)

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.16201377, 0.94292164, 0.1007452 ],
        [0.36029077, 0.2927587 , 0.8291427 ],
        [0.53630257, 0.89399254, 0.09102809],
        ...,
        [0.4168471 , 0.28382587, 0.20251119],
        [0.29758465, 0.6663122 , 0.2495557 ],
        [0.3149761 , 0.6657804 , 0.5853474 ]],

       [[0.54661167, 0.8422831 , 0.16821873],
        [0.792871  , 0.9907869 , 0.6315404 ],
        [0.5799947 , 0.07343316, 0.5707171 ],
        ...,
        [0.5498556 , 0.92442036, 0.14810407],
        [0.68812954, 0.7885026 , 0.9401586 ],
        [0.301224  , 0.9187043 , 0.6071917 ]],

       [[0.547168  , 0.87582195, 0.71346927],
        [0.573351  , 0.38068724, 0.01642704],
        [0.81672776, 0.8458458 , 0.77300787],
        ...,
        [0.53125596, 0.80832124, 0.90029967],
        [0.986446  , 0.5975274 , 0.03262579],
        [0.18004811, 0.18750322, 0.15393353]],

       ...,

       [[0.50895655, 0.27510226, 0.31821847],
        [0.78

In [ ]:
#9
e = random_gen.uniform(shape=(10,), minval=0, maxval=10, dtype=tf.int32)
e, tf.argmax(e), e[tf.argmax(e)]

(<tf.Tensor: shape=(10,), dtype=int32, numpy=array([3, 4, 9, 9, 5, 7, 7, 7, 2, 2], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int32, numpy=9>)

In [ ]:
#10
tf.one_hot(e, depth=10)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

# 📖 00. TensorFlow Fundamentals Extra-curriculum


1. Read through the list of TensorFlow Python APIs, pick one we haven't gone through in this notebook, reverse engineer it (write out the documentation code for yourself) and figure out what it does.

1. Try to create a series of tensor functions to calculate your most recent grocery bill (it's okay if you don't use the names of the items, just the price in numerical form).

1. How would you calculate your grocery bill for the month and for the year using tensors?

1. Go through the TensorFlow 2.x quick start for beginners tutorial (be sure to type out all of the code yourself, even if you don't understand it).


- Are there any functions we used in here that match what's used in there? Which are the same? Which haven't you seen before?


- Watch the video "What's a tensor?" - a great visual introduction to many of the concepts we've covered in this notebook.

In [ ]:
#1

a = tf.constant([1.,2.,3.])
b = tf.constant([1.,2.,3.])
tf.Assert(tf.greater_equal(tf.math.count_nonzero(a==b, dtype=tf.int32), tf.size(a)), [a, b], summarize=tf.size(a))

In [ ]:
#2
import pandas as pd
df = pd.DataFrame([['2021', '01', 100], ['2021', '01', 200], ['2021', '05', 600], ['2021', '05', 900], ['2022', '01', 1000], ['2022', '08', 900], ['2023', '01', 10], ['2023', '01', 80]], columns=['Year', 'Month', 'Expense'])
df['Month'] = pd.Categorical(df['Month'])
categories = list(df.Month.cat.categories)
index_to_category_map = dict(zip(range(len(categories)), categories))
category_to_index_map = dict(zip(categories, range(len(categories))))
df['Month'] = df.Month.cat.codes
df['Year'] = df['Year'].astype(int)


In [ ]:
df_tensor = tf.convert_to_tensor(df.values)
total_bill = tf.reduce_sum(df_tensor[:,2])

In [ ]:
total_bill

<tf.Tensor: shape=(), dtype=int64, numpy=3790>

In [ ]:
#3
year=2022
month_id = '01'
month=category_to_index_map.get(month_id, 13)
cost = tf.reduce_sum(df_tensor[tf.equal(df_tensor[:, 0], year) & tf.equal(df_tensor[:, 1], month)][:,2])
print(f"Total cost in the month of {month_id} {year} = {0 if not cost.numpy() else cost.numpy()}")

Total cost in the month of 01 2022 = 1000


#4 TensorFlow 2.x quick start for beginners tutorial 

- Some deep learning frameworks

  - keras
  - tensorflow
  - pytorch (Python, C++, Java)
  - caffe (Has python installation)
  - theano (Python)
  - dl4j (Java)
  - chainer (Python)

- TF 1.0 vs TF 2.0
  - Eager execution in default mode in TF 2.0 (build and fit simultaneously; no tf.Session)

  - tf.function: Can convert python functions to DAGs JIT

- Hierachy
  - L1 (Top) : Keras, Estimators
  - L2 : Layers, Losses and Metrics
  - L3: Low level APIs (Entensive control)
  - L4: CPU/GPU/TPU

- Architecture
  - Read & Process Data
    - tf.Data, Feature Columns
  - Model building
    - tf.Keras, Premade Estimators, Tensorflow Hub
  - Distribution Strategy
  - Saved Model 
  - Serving 
    - TF Serving
    - TF Lite
    - TF.js on browser
    - Other language bindings on C, Java, Go, C#, etc. 


- Components in TF
  - Constants
  - Variables (memory buffers)
  - Session


- Methods
  - Concatenation
    - tf.concatenate([A, B], axis=0)

  

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [ ]:
A = tf.constant([[4, 3], [6, 1]])

B = tf.Variable([[3, 1], [5, 2]])

A_B_concatenated = tf.concat([A, B], axis=0)

A, B, A_B_concatenated

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[4, 3],
        [6, 1]], dtype=int32)>,
 <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
 array([[3, 1],
        [5, 2]], dtype=int32)>,
 <tf.Tensor: shape=(4, 2), dtype=int32, numpy=
 array([[4, 3],
        [6, 1],
        [3, 1],
        [5, 2]], dtype=int32)>)

In [ ]:
A = tf.zeros(shape=(3,4), dtype=tf.int32)

B = tf.ones(shape=(3,4), dtype=tf.int32)

A, B

(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
 array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
 array([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]], dtype=int32)>)

In [ ]:
A = tf.random.uniform(shape=[3,4])

reshaped_A = tf.reshape(A, shape=[4,3])

A, reshaped_A

(<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[0.24900055, 0.48465526, 0.39165282, 0.8218267 ],
        [0.23590696, 0.39270067, 0.96157587, 0.7378106 ],
        [0.3086977 , 0.7820475 , 0.36411405, 0.43852115]], dtype=float32)>,
 <tf.Tensor: shape=(4, 3), dtype=float32, numpy=
 array([[0.24900055, 0.48465526, 0.39165282],
        [0.8218267 , 0.23590696, 0.39270067],
        [0.96157587, 0.7378106 , 0.3086977 ],
        [0.7820475 , 0.36411405, 0.43852115]], dtype=float32)>)

In [ ]:
A = tf.constant([[4.2, 3.4],
                 [6.5, 1.7],
                 [7.2, 3.9],
                 [6.9, 2.7]])

tensor_A_as_int = tf.cast(A, dtype=tf.int32)

A, tensor_A_as_int

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[4.2, 3.4],
        [6.5, 1.7],
        [7.2, 3.9],
        [6.9, 2.7]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=int32, numpy=
 array([[4, 3],
        [6, 1],
        [7, 3],
        [6, 2]], dtype=int32)>)

In [ ]:
A = tf.constant([[4, 3], [6, 1]])
tensor_A_transpose = tf.transpose(A)

A, tensor_A_transpose

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[4, 3],
        [6, 1]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[4, 6],
        [3, 1]], dtype=int32)>)

In [ ]:
A = tf.constant([[5, 8],
                [3, 9]])

v = tf.constant([[4],
                 [2]])

A, v, A@v, tf.matmul(A, v)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[5, 8],
        [3, 9]], dtype=int32)>, <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[4],
        [2]], dtype=int32)>, <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[36],
        [30]], dtype=int32)>, <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[36],
        [30]], dtype=int32)>)

In [ ]:
A = tf.constant([[5, 8],
                [3, 9]])

v = tf.constant([[4],
                 [2]])

A*v

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 32],
       [ 6, 18]], dtype=int32)>

In [ ]:
A = tf.constant([[4, 9],
                [5, 6],
                [1, 8]])

rows, columns = A.shape
print('rows:', rows, 'columns:', columns)

A_identity = tf.eye(num_rows=rows, num_columns=columns, dtype=tf.int32)
A_identity

rows: 3 columns: 2


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 0],
       [0, 1],
       [0, 0]], dtype=int32)>

In [ ]:
# Demo of building a keras model

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2968 - accuracy: 0.9129
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1434 - accuracy: 0.9570
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1087 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0882 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0743 - accuracy: 0.9773


In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0676 - accuracy: 0.9781 - 943ms/epoch - 3ms/step


[0.06759420037269592, 0.9781000018119812]

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
tf.math.argmax(probability_model(x_test[:5]), axis=1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([7, 2, 1, 0, 4])>

In [ ]:
y_test[:5]

array([7, 2, 1, 0, 4], dtype=uint8)